In [13]:
import pandas as pd
import numpy as np

# Regression Model

In [14]:
import graphlab

In [15]:
hotels = graphlab.SFrame('case_study_data.csv')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1499131285.log


This non-commercial license of GraphLab Create for academic use is assigned to tjcartwright91@gmail.com and will expire on October 18, 2017.


Finished parsing file /Users/thomascartwright/Documents/Development/Data Science/Hotel Data/case_study_data.csv

Parsing completed. Parsed 100 lines in 0.103253 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,int,int,int,int,float,int,float,float,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/thomascartwright/Documents/Development/Data Science/Hotel Data/case_study_data.csv

Parsing completed. Parsed 46647 lines in 0.102453 secs.

First let's remove all the NaN values.

In [16]:
hotels = hotels[hotels['hotel_feature_1'] != None]
hotels = hotels[hotels['hotel_feature_2'] != None]

In [17]:
len(hotels)

45120

Now let's add in our target feature 'price_per_night'

In [18]:
hotels['price_per_night'] = hotels['hotel_price'] / (hotels['departure'] - hotels['arrival'])

Split our data into train and test_data

In [19]:
train_data, test_data = hotels.random_split(.8, seed=0)

In [20]:
print "Number of train_data entries " + str(len(train_data))
print "Number of test data entries "+ str(len(test_data))

Number of train_data entries 36136
Number of test data entries 8984


In [21]:
features = ['search_date', 'arrival',
              'num_adults', 'num_children',
              'hotel_feature_1', 'hotel_feature_2', 
              'hotel_feature_3', 'hotel_feature_4', 
              'hotel_feature_5']
model = graphlab.linear_regression.create(train_data, target='price_per_night', 
                                          features = features, validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 36136

Number of features          : 9

Number of unpacked features : 9

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.027690     | 14200.296712       | 605.715387    |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [22]:
weight_summary = model.get('coefficients')
weight_summary

name,index,value,stderr
(intercept),None,356.567666495,19.0263609987
search_date,None,-1.91551691009,0.436946795373
arrival,None,0.167203667386,0.0357385459306
num_adults,None,36.2014293122,3.59782912124
num_children,None,32.8287170612,3.50348539158
hotel_feature_1,None,-1.43959032756,0.0841638828451
hotel_feature_2,None,-1.35800847066,0.142318842596
hotel_feature_3,None,0.37027882749,0.0652718484914
hotel_feature_4,None,11.0449036328,0.580110757214
hotel_feature_5,None,-10.4594834849,1.83438032357


In [23]:
predictions = model.predict(train_data)

In [24]:
max(predictions)

894.0437756916132

In [25]:
min(predictions)

30.873727366704145

Now let's compute the RSS to evaluate the model's predictions.

### Compute the RSS

In [26]:
def residual_sum_squares(model, data):
    example_predictions = model.predict(data)
    # Then compute the residuals/errors
    errs = example_predictions - data['price_per_night']
    # Then square and add them up
    RSS = sum(errs*errs)

    return(RSS)    

In [27]:
rss_predictions = residual_sum_squares(model, test_data)
rss_predictions

3773664911.830106

As we can see, this RSS is extremely high for the dataset and suggests a poor fitting model.